In [1]:
from tqdm import tqdm
import whisper
from openai import OpenAI
from dataclasses import dataclass
import os
from dotenv import load_dotenv
import requests
import re
import json

load_dotenv()

True

# 初始化模型服务
大多数模型服务适配openai 接口，使用openai sdk可方便的调用各类不同模型服务。  
不同模型服务需配置对应模型服务的API_KEY，BASE_URL，MODEL_NAME   

**_项目.env文件_**   
<img src="./img/env.png" style="width:40%; height:auto;">   
**_智谱文档：https://www.bigmodel.cn/dev/api/thirdparty-frame/openai-sdk_**   
<img src="./img/zpopenaidoc.png" style="width:50%; height:auto;">   
_**deepseek文档：https://api-docs.deepseek.com/zh-cn/_**   
<img src="./img/deepseekdoc.png" style="width:50%; height:auto;">

In [2]:
# 初始化模型配置
@dataclass
class Config:
    """配置管理"""
    API_KEY: str = os.getenv('API_KEY')
    BASE_URL: str = os.getenv('BASE_URL')
    MODEL_NAME: str = os.getenv('MODEL_NAME')
config= Config()
print(f'使用 {config.MODEL_NAME} 模型进行文本处理，base_url:{config.BASE_URL}')

使用 glm-4-plus 模型进行文本处理，base_url:https://open.bigmodel.cn/api/paas/v4/


In [3]:
client = OpenAI(api_key=config.API_KEY, base_url=config.BASE_URL)

# 转录音频
## 本地whisper模型
可使用openai-whisper库加载本地音频识别模型。但存在识别时间长，对方言支持差的缺点。

In [4]:
audio_path = "../audio_dir/白石老人自述.mp3"

In [5]:
%%time
# 加载whisper模型
model = whisper.load_model("small") 
# 转录音频
result = model.transcribe(audio_path)
transcript=result['text']

C:\AccelleronProj\open_memo\.venv\Lib\site-packages\whisper\transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


CPU times: total: 4min 30s
Wall time: 8min 31s


In [7]:
print(transcript)

今天為你介紹的這本書是《白石老人自述》在二十世紀的中國若說最廣為人之雅俗共賞的畫家非《奇白石》末書碧家所稱讚它是東方了不起的畫家周恩來總理更親自授予他人民藝術家的稱號《奇白石》的話兼具民間天然之趣和《奇白石》的名稱《奇白石》的話兼具民間天然之趣和文人藝術的驚威深遠自成一體身手歡迎成就這些優秀畫作的不僅僅是他高超的技藝更是《奇白石》對待生活對待藝術的心性和態度《奇白石》出生於一個窮困的農民家庭一生經歷過晚清民國軍閥割據抗日戰爭等等歷盡坎坷出生和動蕩的時局讓他擁有了很多充滿戲劇性的身份比如不會幹農活的農民喜歡畫畫的木匠苦學作詩的畫工50歲的北漂族和充滿農民氣質的藝術大師關於他的八卦也特別的多比如特別愛錢賣畫從不講價外出吃飯定要打包比如特別喜歡美女83歲生了兒子93歲還在張落取媳婦等等對於自己的一生《奇白石》說世間是貴痛快又自稱草菅偷活說自己是在雜草當中偷偷活下來的小老百姓那麼他究竟是怎麼樣在痛苦與偷活中取得平衡從容應對時代的大變局並從貧苦農民家的孩子修煉成一代大師的呢白石老人自述給我們提供了回答從71歲那年《奇白石》就開始口述自己的生平讓學生張刺西記錄下來準備交給一位叫金松成的作家朋友為他作賺但是賺季未成作家就陰病去世了《奇白石》的口述也斷斷續續直到他89歲的時候才由張刺西最後整理完成在這本書當中《奇白石》回顧了自己從農民到首一人最後成為大畫家的從藝之路也談到了他為人處世和日常生活中的很多細節下面就讓我們一起走進這本書聽白石老人講一講他的畫畫經生活經《奇白石》出生於1864年是湖南湘潭人他們家足足未味都是農民出生的時候家裡邊只有幾間破舊的老屋和一母水田僅能夠勉強虎口少年時的貧窮讓《奇白石》一聲記憶猶新他說我們家平常過日子本已是窮對付一遇到田裡收不多日子就更不好過窮得連糧食都沒得吃了母親就讓我們去田裡挖芋頭野菜用牛糞威著吃後來我每回畫芋頭總會想起當年的情景窮人家的苦滋味只有窮人自己明白雖然家裡很窮但是《奇白石》是在一個充滿愛的氛圍當中長大的它是家裡邊的長子祖父給他取名為奇純芝靈芝的芝小名阿芝阿芝深得祖父喜愛冬天冷的時候祖父會穿上最好的羊皮好敞開衣巾把阿芝揉在懷裡然後用火鉗子劃著盧輝教他韌字在祖父的耐心教導下阿芝先後十了三百多個字有了基本的毒血能力阿芝在三歲之前經常生病但是家裡窮買不起藥祖母和母親就每天去藥鋪求情有個大夫說身體弱的小孩最好不要沾染婚姓那個時候阿芝還在吃母乳母

## 阶跃星辰音频转写模型服务
利用商业api服务提升音频转录质量。   
这里以阶跃星辰step-asr为例。(https://github.com/stepfun-ai/Step-Audio, 模型具备方言识别能力)   
接口文档：https://platform.stepfun.com/docs/api-reference/audio/transcriptions

In [8]:
def transcribe_audio(audio_path: str) -> dict:
    step_key = os.getenv('STEP_KEY')
    url = 'https://api.stepfun.com/v1/audio/transcriptions'
    headers = {
        'Authorization': f'Bearer {step_key}',
    }
    with open(audio_path, 'rb') as audio_file:
        files = {
                'file': audio_file,
            }
        data={'model': 'step-asr', 'response_format': 'json'}
        response = requests.post(url, headers=headers, files=files,data=data)
        return response.json()

In [9]:
%%time
response = transcribe_audio(audio_path)
transcript=response['text']
print(transcript)

今天为你介绍的这本书是《白石老人自述》。在二十世纪的中国，若说最广为人知雅俗共赏的画家非齐白石莫属。毕加索称赞他是东方了不起的画家，周恩来总理更亲自授予他人民艺术家的称号。齐白石的画兼具民间天然之趣和文人艺术的精微深远，自成一体，深受欢迎。成就这些优秀画作的不仅仅是他高超的技艺，更是齐白石对待生活，对待艺术的心性和态度。齐白石出生于一个穷困的农民家庭，一生经历过晚清、民国、军阀割据、抗日战争等等，历尽坎坷。出生和动荡的时局让他拥有了很多充满戏剧性的身份，比如不会干。农活的农民，喜欢画画的木匠，苦学作诗的画工，五十岁的北漂族和充满农民气质的艺术大师。关于他的八卦也特别的多，比如，特别爱钱，卖画，从不讲价，外出吃饭定要打包，比如，特别喜欢美女，八十三岁生了儿子，九十三岁还在张罗娶媳妇，等等。对于自己的一生，齐白石说，世间事，贵痛快，又自称草间偷活，说自己啊，是在杂草当中偷偷活下来的小老百姓，那么，他究竟是怎么样在痛苦与偷活中取得平衡，从容应对时代的大变局，并从贫苦农民家的孩子修炼成一代大师的呢？这本白石老人字，给我们提供了回答。从七十一岁那年，齐白石就开始口述自己的生平，让学生张次希记录下来，准备交给一位叫金松岑的作家朋友为他作传，但是传记未成，作家就因病去世了。齐白石的口述也断断续续，直到他八十九岁的时候，才由张次希最后整理完成。在这本书当中呢，齐白石回顾了自己从农民到手艺人，最后成为大画家的从艺之路，也谈到了他为人处事和日常生活中的很多细节。下面，就让我们一起走进这本书，听白石老人讲一讲他的画画经，生活经。齐白石出生于一八六四年，是湖南湘潭人，他们家祖祖辈辈都是农民。出生的时候，家里边只有几间破旧的老屋和一亩水田，仅能够勉强糊口。少年时的贫穷让齐白石一生记忆犹新。他说，我们家平常过日子，本已是穷对付，一遇到田里收不多，日子就更不好过，穷得连粮食都没得吃了。母亲就让我们去田里挖芋头，野菜用牛粪煨着吃。后来我每回画芋头总会想起当年的情景，穷人家的苦滋味只有穷人自己明白。虽然家里很穷，但是齐白石是在一个充满爱的氛围当中长大的。他是家里边的长子，祖父给他取名为齐纯之，灵之的之，小名阿之。阿芝深得祖父喜爱。冬天冷的时候，祖父会穿上最好的羊皮袄，敞开衣襟，把阿芝搂在怀里，然后用火钳子画着炉灰教她认字。在祖父的耐心教导下，阿芝先后识了三百多个字，有了基本的读写能力。阿芝在三岁之前

# 分析故事结构

## prompt工程1：故事结构分析指令 

In [10]:
analysis_prompt="""# 角色
你是一名专业文学分析师，擅长多故事解构和关键信息提炼。具备精准识别叙事单元、解析时空逻辑、提取核心要素的能力。

## 任务说明
### 核心功能：
对输入的复合文本进行深度解析，完成：
1. 独立叙事单元的识别与分割
2. 故事要素的精准提取
3. 结构化数据输出

## 技能树
### 技能 1：故事拆分
1. 逐句分析文本，识别时间/地点/人物的非连续突变
2. 通过叙事断层检测判断独立故事数量
3. 确定每个故事的起止边界（承转合节点）

### 技能 2：结构化输出
1. 为每个故事创建独立数据对象
2. 要素提取标准：
    - 标题：基于核心矛盾提炼（中性表述）
    - 时间：精确到最小可识别单位（例："工业革命初期"→"1760年代"）
    - 人物：排除次要人物（出场＜3次/无关键行动）
    - 摘要：包含「冲突起源-关键转折-结局」三要素

### 技能 3：格式规范
1. 严格遵循以下的JSON架构，不要添加任何其他额外内容：
{
    "stories": [
        {
            "story_id": 1,
            "story_title": "标题",
            "story_time": "时间描述",
            "characters": ["人物1", "人物2"],
            "summary": "摘要内容"
        }
    ]
}
2. 摘要长度控制：
    - 中文不超过200字（含标点）
    - 英文不超过400字符

## 约束条件
1. 必须直接输出合法JSON，不要添加任何额外说明
2. 绝对禁止合并具有时空断层的故事
3. 时间解析必须满足：
    - 明确时间＞模糊时间（原文"春天"需保持模糊）
    - 拒绝推断不存在的时间信息
4. 人物字段需去重处理


## 错误处理
1. 当检测到非故事内容（论述/说明/对话）时：
    - 丢弃非叙事段落
    - 返回错误类型："NON_STORY_CONTENT"
2. 当出现无法解析的时间信息时：
    - 保留原文表述
    - 添加注释："TIME_AMBIGUITY"     
"""

## 生成故事结构(json结构化数据)

In [11]:
response = client.chat.completions.create(
                    model=config.MODEL_NAME,
                    messages=[
                        {"role": "system", "content": analysis_prompt},
                        {"role": "user", "content": f"请分析这段内容：{transcript}"}
                    ],
                    temperature=0.3
                )

通过在prompt中指定文本数据结构，获得json格式文本，以用于衔接后续工作流

In [12]:
def parse_markdown_json(markdown_text):
    # 匹配包含 JSON 的 Markdown 代码块
    pattern = r'```json\n(.*?)```'
    match = re.search(pattern, markdown_text, re.DOTALL)

    if not match:
        print("Error: 找不到 JSON 代码块")
        return None

    json_str = match.group(1).strip()

    try:
        # 解析 JSON 字符串为 Python 对象
        return json.loads(json_str)
    except json.JSONDecodeError as e:
        print(f"JSON 解析失败: {e}")
        return None

In [13]:
story_data=parse_markdown_json(response.choices[0].message.content)
story_data

{'stories': [{'story_id': 1,
   'story_title': '齐白石的生平与艺术成就',
   'story_time': '1864-1957',
   'characters': ['齐白石', '胡庆元', '陈诗从', '胡宝珠', '夏文珠'],
   'summary': '齐白石，湖南湘潭人，生于1864年，历经晚清、民国、军阀割据、抗日战争等动荡时期。从贫困农民家庭出身，通过自学成为著名画家。他的画作兼具民间趣味和文人艺术精微，形成独特风格。齐白石一生勤奋，日日作画，不断创新，最终成为一代艺术大师。他的生活法则包括重视金钱、注重礼数、保持豁达心态和对美人的热爱。'},
  {'story_id': 2,
   'story_title': '齐白石的艺术之路',
   'story_time': '1889-1957',
   'characters': ['齐白石', '胡庆元', '陈诗从'],
   'summary': '齐白石从木匠起步，通过自学和拜师，逐步成为画家。他深受《芥子园画谱》影响，打下扎实基础。在胡庆元的指导下，正式开始学画。经过多次远游，开阔眼界，作品风格发生质变。在北京时期，他听从陈诗从建议，改变画法，创造出红花墨叶画法，大获成功，成为画坛泰斗。'},
  {'story_id': 3,
   'story_title': '齐白石的生活智慧',
   'story_time': '1864-1957',
   'characters': ['齐白石', '陈春君', '胡宝珠', '夏文珠'],
   'summary': '齐白石在生活中注重金钱、礼数和心态。他直接坦率地卖画，保持节俭习惯。待人处事有明确标准，但也不拘泥于礼数。他心胸豁达，专注提升自己，不过问分外事。对美人充满热爱，一生多段感情经历，但对待每段感情都真诚坦荡。'}]}

# 生成故事

## prompt工程2： 生成故事

In [14]:
def _get_story_prompt(story_info: dict) -> str:
    """获取故事生成提示词"""
    stories=story_info.get('stories')
    contents=''
    for story in stories:
        content=f"""
    标题：{story['story_title']}
    时间：{story['story_time']}
    人物：{', '.join(story['characters'])}
    梗概：{story['summary']}"""
        contents+=content+'\n'
    
    return f"""请基于以下信息，编写一个完整的故事。请直接输出故事内容，不要包含任何格式标记：
    
    {contents}
    
    要求：
    1. 保持故事的完整性和连贯性
    2. 添加适当的环境描写和细节
    3. 展现人物的性格特征和情感变化
    4. 故事长度控制在1000字左右
    5. 运用报告文学的语言风格"""

In [15]:
prompt=_get_story_prompt(story_data)
print(prompt)

请基于以下信息，编写一个完整的故事。请直接输出故事内容，不要包含任何格式标记：
    
    
    标题：齐白石的生平与艺术成就
    时间：1864-1957
    人物：齐白石, 胡庆元, 陈诗从, 胡宝珠, 夏文珠
    梗概：齐白石，湖南湘潭人，生于1864年，历经晚清、民国、军阀割据、抗日战争等动荡时期。从贫困农民家庭出身，通过自学成为著名画家。他的画作兼具民间趣味和文人艺术精微，形成独特风格。齐白石一生勤奋，日日作画，不断创新，最终成为一代艺术大师。他的生活法则包括重视金钱、注重礼数、保持豁达心态和对美人的热爱。

    标题：齐白石的艺术之路
    时间：1889-1957
    人物：齐白石, 胡庆元, 陈诗从
    梗概：齐白石从木匠起步，通过自学和拜师，逐步成为画家。他深受《芥子园画谱》影响，打下扎实基础。在胡庆元的指导下，正式开始学画。经过多次远游，开阔眼界，作品风格发生质变。在北京时期，他听从陈诗从建议，改变画法，创造出红花墨叶画法，大获成功，成为画坛泰斗。

    标题：齐白石的生活智慧
    时间：1864-1957
    人物：齐白石, 陈春君, 胡宝珠, 夏文珠
    梗概：齐白石在生活中注重金钱、礼数和心态。他直接坦率地卖画，保持节俭习惯。待人处事有明确标准，但也不拘泥于礼数。他心胸豁达，专注提升自己，不过问分外事。对美人充满热爱，一生多段感情经历，但对待每段感情都真诚坦荡。

    
    要求：
    1. 保持故事的完整性和连贯性
    2. 添加适当的环境描写和细节
    3. 展现人物的性格特征和情感变化
    4. 故事长度控制在1000字左右
    5. 运用报告文学的语言风格


In [16]:
response = client.chat.completions.create(
                    model=config.MODEL_NAME,
                    messages=[
                        {"role": "system", "content": prompt},
                        {"role": "user", "content": "你是一个专业的故事创作者，请直接输出故事内容。"}
                    ],
                    temperature=0.3
                )

In [17]:
print(response.choices[0].message.content)

齐白石，这位从湖南湘潭走出的艺术大师，生于1864年，一生历经晚清、民国、军阀割据、抗日战争等动荡时期。他的艺术之路，既是一部个人奋斗史，也是中国近现代艺术发展的缩影。

齐白石出身于一个贫困的农民家庭，早年以木匠为生。1889年，一个偶然的机会，他得到了一本《芥子园画谱》，从此便开始了自学绘画的历程。那时的他，白天挥汗如雨地干着木匠活，夜晚则在昏暗的油灯下临摹画谱，逐渐打下扎实的绘画基础。

在胡庆元的指导下，齐白石正式踏上了学画之路。胡庆元是一位颇有名气的画家，他看中了齐白石的勤奋和天赋，悉心传授技艺。齐白石也不负所望，刻苦钻研，技艺日渐精进。经过多次远游，他开阔了眼界，作品风格发生了质变，逐渐形成了兼具民间趣味和文人艺术精微的独特风格。

1917年，齐白石来到北京，开始了他的艺术生涯的新篇章。在北京，他结识了陈诗从，这位艺术评论家对齐白石的画法提出了中肯的建议。陈诗从认为，齐白石的画作虽功底深厚，但缺乏创新。齐白石虚心接受，经过反复试验，终于创造出红花墨叶画法，这一创新使他的作品大获成功，他也因此成为画坛泰斗。

齐白石不仅在艺术上成就斐然，他的生活智慧也令人称道。他注重金钱，但并非贪婪，而是深知生活的艰辛。他直接坦率地卖画，保持节俭习惯，从不挥霍浪费。待人处事，他有着明确的标准，但也不拘泥于礼数，心胸豁达，专注提升自己，不过问分外事。

在感情生活中，齐白石同样展现出真诚坦荡的一面。他与原配陈春君相濡以沫，尽管后来又有了胡宝珠和夏文珠等多段感情经历，但他对待每段感情都极为认真，从不敷衍。胡宝珠是他艺术生涯中的重要伴侣，不仅在生活中给予他无微不至的照顾，还在艺术上给予他诸多启发。夏文珠则在他晚年时陪伴左右，给予他精神上的慰藉。

齐白石的一生，是勤奋与创新的典范。他日日作画，不断创新，即使在晚年也未曾懈怠。他的画作，既有深厚的传统功底，又充满鲜活的生活气息，成为一代艺术大师。他的生活智慧，更是为后人提供了宝贵的人生启示。

1957年，齐白石走完了他传奇般的一生，享年九十三岁。他的艺术成就和生活智慧，至今仍为人们所津津乐道。齐白石的故事，不仅是一段个人奋斗的历史，更是一段中国近现代艺术发展的辉煌篇章。
